In [1]:
import os 
import nibabel as nib
import numpy as np
import pandas as pd

# <span style="color:#FFCC80">In vivo Swiss data processing </span> </br>
Compare and analyze if using optimized parameters improves or deteriorates the final local fields </br>

Keywords: </br>
* sw = slicewise
* pp = pipeline

In [2]:
root_dir = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing"

In [14]:
# We have 4 subjects inside
subjects = ['hc1', 'hc2', 'sci1', 'sci2']
measurements = ['m1', 'm2', 'm3', 'm4']
bgfr_algos = ['opt_pdf', 'opt_resharp', 'opt_sharp']
rows = []


In [32]:
# We have 4 subjects inside
subjects = ['hc1', 'hc2'] #, 'sci1', 'sci2']
measurements = ['m1', 'm2', 'm3', 'm4']
bgfr_algos = ['opt_lbv', 'opt_pdf', 'opt_resharp', 'opt_sharp', 'opt_resharp/dilated_mask', 'opt_sharp/dilated_mask']#,'opt_vsharp_8_to_1','def_iHarperella']
lf_rows = []

In [33]:
# Now lets collect rows for LF!

for subj in subjects:
    for meas in measurements:
        for algo in bgfr_algos:
            
            # Firsst, define the path pointers
            subj_meas_path = os.path.join(root_dir, subj, meas)
            lf_path = os.path.join(subj_meas_path, 'local_field', algo,'Sepia_localfield.nii.gz')
            if lf_path is None or not os.path.exists(lf_path):
                print(f"Skipping {lf_path} as it does not exist.")
                continue
            gm_msk_path = os.path.join(subj_meas_path, f'{subj}_{meas}_gm_mks.nii.gz')
            wm_msk_path = os.path.join(subj_meas_path, f'{subj}_{meas}_wm_msk.nii.gz')

            # Load data
            lf_img = nib.load(lf_path)
            lf_data = lf_img.get_fdata()

            gm_mask = nib.load(gm_msk_path).get_fdata()
            total_vox_gm = np.sum(gm_mask==1) 
            wm_mask = nib.load(wm_msk_path).get_fdata()
            total_vox_wm = np.sum(wm_mask==1)

            # Compute metrics for GM and WM
            gm_mean = np.mean(lf_data[gm_mask==1])
            gm_std = np.std(lf_data[gm_mask==1])

            wm_mean = np.mean(lf_data[wm_mask==1])
            wm_std = np.std(lf_data[wm_mask==1])

            # Compute how many voxels are in the mask
            gm_nonzero_vox = np.sum(lf_data[gm_mask==1] != 0)
            wm_nonzero_vox = np.sum(lf_data[wm_mask==1] != 0)

            # Compute the contrast metric
            # We want to maximize the contrast between GM and WM
            # WM mean should be negative, if it is positive the contrast will be lower this way
            contrast = np.abs(gm_mean - wm_mean)
            normalizer_denominator = np.sqrt(gm_std**2 + wm_std**2)
            raw_metric = contrast / normalizer_denominator if normalizer_denominator != 0 else 0 # Just in case that the std is 0 - to avoid division by zero

            # Now we penalize if the algo eroded the mask
            gm_penality = gm_nonzero_vox / total_vox_gm 
            wm_penality = wm_nonzero_vox / total_vox_wm

            # Final metric:
            final_metric = raw_metric * gm_penality * wm_penality

            # Now collect row and add to data frame
            lf_rows.append({
                'subject': subj,
                'measurement': meas,
                'algo': algo,
                'mean_gm': gm_mean,
                'std_gm': gm_std,
                'total_vox_gm': total_vox_gm,
                'nonzero_vox_gm': gm_nonzero_vox,
                'mean_wm': wm_mean,
                'std_wm': wm_std,
                'total_vox_wm': total_vox_wm,
                'nonzero_vox_wm': wm_nonzero_vox,
                'contrast factor': contrast,
                'gm_penality': gm_penality,
                'wm_penality': wm_penality,
                'raw_metric': raw_metric,
                'final_metric': final_metric
            })


# Create the data frame
lf_df = pd.DataFrame(lf_rows)

# Save
lf_df.to_csv(os.path.join(root_dir, 'bgfr_metrics_complete_dubs_and_meass_dilated_test.csv'), index=False)
print(lf_df)

Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m1\local_field\opt_resharp/dilated_mask\Sepia_localfield.nii.gz as it does not exist.
Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m1\local_field\opt_sharp/dilated_mask\Sepia_localfield.nii.gz as it does not exist.
Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m2\local_field\opt_resharp/dilated_mask\Sepia_localfield.nii.gz as it does not exist.
Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m2\local_field\opt_sharp/dilated_mask\Sepia_localfield.nii.gz as it does not exist.
Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m3\local_field\opt_resharp/dilated_mask\Sepia_localfield.nii.gz as it does not exist.
Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m3\local_field\opt_sharp/dilated_mask\Sepia_localfield.nii.gz as it does not exist.
Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m4\local_field\opt_resharp/dilated_mask\Sepia_localfield

# <span style="color:#16D8E6">Loading and showing the winners</span> </br>

In [21]:
examination_csv_hc1_hc2 =  'bgfr_metrics_complete_dubs_and_meass.csv' # Select the csv to read
lf_df_hc1_hc2 = pd.read_csv(os.path.join(root_dir,examination_csv_hc1_hc2))
ld_df_hc1_hc2_sorted = lf_df_hc1_hc2.sort_values(by='final_metric', ascending=False)

In [30]:
ld_df_hc1_hc2_sorted.head(5)

,subject,measurement,algo,mean_gm,std_gm,total_vox_gm,nonzero_vox_gm,mean_wm,std_wm,total_vox_wm,nonzero_vox_wm,contrast factor,gm_penality,wm_penality,raw_metric,final_metric
17,hc2,m1,opt_pdf,0.378919,0.470250,671,670,-0.079148,0.436276,3264,3264,0.458067,0.998510,1.0,0.714099,0.713035
21,hc2,m2,opt_pdf,0.341338,0.444211,685,683,-0.071059,0.417333,3240,3240,0.412397,0.997080,1.0,0.676616,0.674640
29,hc2,m4,opt_pdf,0.288127,0.457951,700,697,-0.068618,0.421181,3254,3254,0.356745,0.995714,1.0,0.573375,0.570918
25,hc2,m3,opt_pdf,0.244376,0.462340,653,651,-0.046069,0.413561,3265,3265,0.290445,0.996937,1.0,0.468222,0.466788
5,hc1,m2,opt_pdf,0.312296,0.754002,647,644,-0.058237,0.689719,3362,3362,0.370532,0.995363,1.0,0.362600,0.360919


In [34]:
dilated_test =  'bgfr_metrics_complete_dubs_and_meass_dilated_test.csv' # Select the csv to read
lf_df_hc1_hc2_dil_test= pd.read_csv(os.path.join(root_dir,dilated_test))
ld_df_hc1_hc2_dil_test_sorted= lf_df_hc1_hc2_dil_test.sort_values(by='final_metric', ascending=False)

In [35]:
ld_df_hc1_hc2_dil_test_sorted.head(5)

,subject,measurement,algo,mean_gm,std_gm,total_vox_gm,nonzero_vox_gm,mean_wm,std_wm,total_vox_wm,nonzero_vox_wm,contrast factor,gm_penality,wm_penality,raw_metric,final_metric
17,hc2,m1,opt_pdf,0.378919,0.470250,671,670,-0.079148,0.436276,3264,3264,0.458067,0.998510,1.000000,0.714099,0.713035
23,hc2,m2,opt_pdf,0.341338,0.444211,685,683,-0.071059,0.417333,3240,3240,0.412397,0.997080,1.000000,0.676616,0.674640
20,hc2,m1,opt_resharp/dilated_mask,0.389559,0.756573,671,671,-0.313362,0.964124,3264,3262,0.702921,1.000000,0.999387,0.573562,0.573211
31,hc2,m4,opt_pdf,0.288127,0.457951,700,697,-0.068618,0.421181,3254,3254,0.356745,0.995714,1.000000,0.573375,0.570918
27,hc2,m3,opt_pdf,0.244376,0.462340,653,651,-0.046069,0.413561,3265,3265,0.290445,0.996937,1.000000,0.468222,0.466788


In [37]:
ld_df_hc1_hc2_dil_test_sorted_contrast = lf_df_hc1_hc2_dil_test.sort_values(by='contrast factor', ascending=False)
ld_df_hc1_hc2_dil_test_sorted_contrast.head(5)

,subject,measurement,algo,mean_gm,std_gm,total_vox_gm,nonzero_vox_gm,mean_wm,std_wm,total_vox_wm,nonzero_vox_wm,contrast factor,gm_penality,wm_penality,raw_metric,final_metric
20,hc2,m1,opt_resharp/dilated_mask,0.389559,0.756573,671,671,-0.313362,0.964124,3264,3262,0.702921,1.000000,0.999387,0.573562,0.573211
0,hc1,m1,opt_lbv,0.530615,0.589751,645,438,0.058260,0.342314,3375,1300,0.472355,0.679070,0.385185,0.692707,0.181190
17,hc2,m1,opt_pdf,0.378919,0.470250,671,670,-0.079148,0.436276,3264,3264,0.458067,0.998510,1.000000,0.714099,0.713035
10,hc1,m3,opt_resharp,0.382815,3.171093,623,593,-0.052051,1.555972,3413,1770,0.434866,0.951846,0.518605,0.123113,0.060772
1,hc1,m1,opt_pdf,0.348916,1.140037,645,645,-0.067127,0.753490,3375,3375,0.416042,1.000000,1.000000,0.304449,0.304449
